In [1]:
# Import libraries
import numpy as np 
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Prepare Dataset
# load data
filename_1 = 'output_batch_1.npz'
filename_0 = 'output_batch_0.npz'
filename_2 = 'output_batch_2.npz'
filename_3 = 'output_batch_3.npz'
filename_4 = 'output_batch_4.npz'
filename_5 = 'output_batch_5.npz'
filename_6 = 'output_batch_6.npz'
filename_8 = 'output_batch_8.npz'
filename_9 = 'output_batch_9.npz'
filename_10 = 'output_batch_10.npz'
filename_11 = 'output_batch_11.npz'
folder_path = 'Measurements'
current_directory = 'c:/Users/20201558/Desktop/Work related stuff/Master/Year 4/Quartile 3/Interdisciplinary Team Project [5ARIP10]/Code/CNN-RNN'
file_path_0 = os.path.join(current_directory,folder_path, filename_0)
file_path_1 = os.path.join(current_directory,folder_path, filename_1)
file_path_2 = os.path.join(current_directory,folder_path, filename_2)
file_path_3 = os.path.join(current_directory,folder_path, filename_3)
file_path_4 = os.path.join(current_directory,folder_path, filename_4)
file_path_5 = os.path.join(current_directory,folder_path, filename_5)
file_path_6 = os.path.join(current_directory,folder_path, filename_6)
file_path_8 = os.path.join(current_directory,folder_path, filename_8)
file_path_9 = os.path.join(current_directory,folder_path, filename_9)
file_path_10 = os.path.join(current_directory,folder_path, filename_10)
file_path_11 = os.path.join(current_directory,folder_path, filename_11)
data_0 = np.load(file_path_0)
data_1 = np.load(file_path_1)
data_2 = np.load(file_path_2)
data_3 = np.load(file_path_3)
data_4 = np.load(file_path_4)
data_5 = np.load(file_path_5)
data_6 = np.load(file_path_6)
data_8 = np.load(file_path_8)
data_9 = np.load(file_path_9)
data_10 = np.load(file_path_10)
#data_11 = np.load(file_path_11)
#data = [*data_0,*data_1,*data_2,*data_3,*data_4,*data_5,*data_6,*data_8,*data_9,*data_10]
data = data_0

In [3]:
frames = data['frames']
forces_data = data['forces']
# train test split. Size of train data is 80% and size of test data is 20%. 
features_train, features_test, targets_train, targets_test = train_test_split(frames,
                                                                             forces_data[:,2],
                                                                             test_size = 0.2,
                                                                             random_state = 42) 
# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
featuresTest = torch.from_numpy(features_test)
targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor) # data type is long

In [4]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_channels, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(input_channels, 30, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        #self.conv2 = nn.Conv2d(30, 60, kernel_size=5, stride=1, padding=2)
        
        # RNN
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(30 * 135 * 240, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Convolutional layers
        x = self.pool(nn.ELU()(self.conv1(x)))
        #x = self.pool(nn.ELU()(self.conv2(x)))
        # Reshape for RNN
        x = torch.reshape(x, (-1, 1, 30 * 135 * 240))  # Reshape to (batch_size, seq_len, input_size)
        print("Size of x:", x.size())  # Print size of x
        # RNN
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
        print("Size of h0:", h0.size())  # Print size of h0
        out, hn = self.rnn(x, h0)
        
        # Output layer
        out = self.fc(out[:, -1, :]) 
        return out


In [5]:
# batch_size, epoch and iteration
batch_size = 800
n_iters = 5
num_epochs = n_iters / (len(features_train) / batch_size)
num_epochs = int(num_epochs)

# Pytorch train and test sets
train = TensorDataset(featuresTrain,targetsTrain)
test = TensorDataset(featuresTest,targetsTest)

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_channels = 3  # RGB channels
hidden_dim = 100  # hidden layer dimension
layer_dim = 1     # number of hidden layers
output_dim = 1   # output dimension

model = RNNModel(input_channels, hidden_dim, layer_dim, output_dim)

# Define your loss function
error = nn.MSELoss()

# Define your optimizer
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Training
seq_dim = 50 # Consider the whole batch to be temporally correlated
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.float()
        print(images.shape)  # Add this line to check the shape of images
        images = images.permute(0, 3, 1, 2)
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(images)

        # Calculate loss
        loss = error(outputs, labels.float())
        
        # Backpropagation
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
            
        # Store loss and iteration
        loss_list.append(loss.data)
        if count % 500 == 0:
            # Print Loss
            print('Iteration: {}  Loss: {}'.format(count, loss.data.item()))

torch.Size([800, 270, 480, 3])
Size of x: torch.Size([800, 1, 972000])
Size of h0: torch.Size([1, 800, 100])


c:\ProgramData\Anaconda3\envs\my-env4\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([800])) that is different to the input size (torch.Size([800, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6220800000 bytes.

Progress Summary: Running ended at backwards propagation, most probably due to memory issues. The hyperparameters and layers of the CNN must be changed in order to reduce the number of input features in the RNN (currently 30 x 135 x 240), i.e. not overload the computer. Still, it reaching this point with correct partitioning of data and correct passing through the network indicates that this code could be used to obtain the first results with a slightly different network structure.